In [23]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
df = pd.read_csv('20191226-reviews.csv')

In [6]:
df['helpfulVotes'].fillna(0, inplace=True)
df.dropna(subset=['body'], inplace=True)
df['date']= pd.to_datetime(df['date'])
df.drop('asin', axis=1, inplace=True)
df.drop('name', axis=1, inplace=True)
df.drop('verified', axis=1, inplace=True)
df.drop('title', axis=1, inplace=True)


In [8]:
df['date'].min()

Timestamp('2003-11-24 00:00:00')

In [11]:
from datetime import datetime
start_date = datetime(2003, 11, 24)
df['days_since_start'] = (df['date'] - start_date).dt.days

In [13]:
df.drop('date', axis=1, inplace=True)

In [16]:
sia = SentimentIntensityAnalyzer()
df['pos_words'] = df.apply(lambda x: len([w for w in nltk.word_tokenize(x['body']) if sia.polarity_scores(w)['pos'] > 0]), axis=1)
df['neg_words'] = df.apply(lambda x: len([w for w in nltk.word_tokenize(x['body']) if sia.polarity_scores(w)['neg'] > 0]), axis=1)

In [17]:
df

,rating,body,helpfulVotes,days_since_start,pos_words,neg_words
0,3,I had the Samsung A600 for awhile which is abs...,1.0,687,17,13
1,1,Due to a software issue between Nokia and Spri...,17.0,44,5,2
2,5,"This is a great, reliable phone. I also purcha...",5.0,36,6,3
3,3,"I love the phone and all, because I really did...",1.0,115,3,0
4,4,The phone has been great for every purpose it ...,1.0,643,6,3
...,...,...,...,...,...,...
67981,5,I love the camera on this phone. The screen is...,1.0,5744,5,3
67982,5,I've been an Xperia user for several years and...,1.0,5773,4,5
67983,5,buy one more for my cousin,0.0,5711,0,0
67984,5,Product looks and works like new. Very much re...,0.0,5874,2,0


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df.loc[:, ['helpfulVotes', 'days_since_start', 'pos_words', 'neg_words']], df.loc[:, 'rating'], train_size=0.7, random_state=42)

In [103]:
prm = Pipeline([
    ("poly", PolynomialFeatures(degree=4, include_bias=False)),
    ('linreg', LinearRegression())
])

In [104]:
prm.fit(X_train, y_train)

Pipeline(steps=[('poly', PolynomialFeatures(degree=4, include_bias=False)),
                ('linreg', LinearRegression())])

In [105]:
def predict_rating(body, date, helpfulVotes=0):
    return np.array([helpfulVotes, 
                     (date - start_date).days, 
                     len([w for w in nltk.word_tokenize(body) if sia.polarity_scores(w)['pos'] > 0]),
                     len([w for w in nltk.word_tokenize(body) if sia.polarity_scores(w)['neg'] > 0])
                    ])

In [106]:
predict_rating("I love Gary Numan's older stuff and this is a great tune!!", datetime(2016, 9, 16))[None, :]

array([[   0, 4680,    2,    0]])

In [107]:
prm.predict(predict_rating("Gary Numan's older stuff is ok!!", datetime(2016, 9, 16), 3)[None, :])

array([3.90040203])

In [108]:
prm.named_steps['linreg'].coef_, prm.named_steps['linreg'].intercept_

(array([-7.83324476e-02, -3.47025921e-03, -3.16044586e-02, -1.91652236e-02,
        -5.01278461e-04,  8.82466400e-05, -3.01562214e-03,  1.24280832e-03,
         2.18885461e-06,  1.17068148e-04, -2.14281633e-04,  7.15668089e-04,
        -1.60134399e-02,  3.64292665e-02, -8.17573218e-07,  3.64492570e-07,
         1.21497989e-05, -5.80577761e-05, -3.31082268e-08,  9.40452479e-07,
         3.25417473e-06,  7.47476802e-06, -2.89069749e-05, -1.62857187e-04,
        -4.82045105e-10, -2.08049503e-09,  5.51662584e-09, -5.37912696e-06,
         1.26150473e-05, -3.67411139e-06,  1.26884748e-04, -1.45496530e-04,
         1.87483449e-04, -1.06825963e-03,  2.72084888e-10,  6.97710804e-11,
         2.74009947e-09,  5.40353635e-10, -4.22806859e-11, -2.93819070e-09,
         9.64892821e-09,  9.40881089e-09,  2.66921245e-07, -2.74126475e-07,
         3.32958314e-12, -2.18507734e-11, -5.35813760e-10,  1.37233370e-09,
        -2.86282973e-08,  4.25489559e-08, -1.62163088e-07, -5.24384631e-08,
         2.5

In [109]:
prm.score(X_train, y_train)

0.24805607931666762

In [110]:
prm.score(X_val, y_val)

0.16624319187340542

In [111]:
labels_train = np.where(y_train >= 3.5, "pos", "neg")
labels_val = np.where(prm.predict(X_train) >= 3.5, "pos", "neg")
(labels_train == labels_val).mean()

0.7646873357856017